&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; <b>YOLO Loss Function</b>

<img src="https://i.stack.imgur.com/bQAdd.jpg">

### Set up

In [1]:
import numpy as np
import pandas as pd
import keras.backend as K

Using TensorFlow backend.


In [2]:
img_height = 448
img_width = 448
S = 7
B = 2
C = 2
grid_cell_h = img_height/S
grid_cell_w = img_width/S

In [3]:
a = 0
b = 0
grid_x = []
grid_y = []
for i in range(S):
    grid_x.append([a, a+img_height/S])
    grid_y.append([b, b+img_width/S])
    a += img_height/S
    b += img_width/S

In [4]:
y_pred = np.random.rand(S, S, (5*B+C))
y_true = np.zeros((S, S, 5*B+C))
y_true[6, 0, :5] = np.random.rand(5,)
y_true[6, 0, -C:] = np.random.rand(C,)
y_true[1, 6, :] = np.random.rand(5*B+C,)
y_true[4, 2, :] = np.random.rand(5*B+C,)
# y_pred[6, 0, :] = np.random.rand(7,)
# y_pred[0, 6, :] = np.random.rand(7,)

<pre>third dimension of y_true and y_pred: 5*B+C
 - last C values are for classes
 - order of each Box values: x, y, w, h, C 
</pre>

### Experiments

In [25]:
i = 0
t = (np.sum(y_true[:, :, i*5:(i+1)*5], axis=2)>0).astype(int)
stacked = np.stack([t for ind in range(5)], axis=2)
stacked.shape

np.sum(stacked, axis=2)

(7, 7, 5)

In [45]:

I_1  = np.zeros((S,S,5*B+C), dtype='uint8')# i,j
for i in range(B):
    t = (np.sum(y_true[:, :, i*5:(i+1)*5], axis=2)>0).astype(int)
    stacked = np.stack([t for ind in range(5)], axis=2)
    I_1[:, :, i*5:(i+1)*5] = stacked

(I_1*y_true).shape

(7, 7, 12)

In [46]:
I_1_noobj = np.logical_not(I_1).astype(int)
I_1_noobj[:, :, -C:] = np.zeros((S,S,2))

I_1_noobj.shape

(7, 7, 12)

In [55]:
a = y_true[:, :, -C+1:]
b = (np.sum(a, axis=2)>0).astype(int)
I_2 = np.stack([b for ind in range(C)], axis=2)

In [40]:
B = 2
# y_true = np.zeros((S, S, 5*B))
C_ = list(range(4, 5*B+1, 5))
print(C_) 
I__ = y_true[:, :, C_]
I__.shape

C_ = np.array([4])
C_ = [j+i*5 for i in range(B) for j in C_]
C_

[4, 9]


[4, 9]

In [120]:
x_ = np.array([0,1])
x__ = []
for i in range(B):
    x__.extend(x_+i*5)
    
# y_true[:, :, B:2]
x__
y_true[:, :, x__].shape
a_ = y_pred[:, :, x__]
b_ = y_true[:, :, x__]
np.sum(((a_-b_)**2)*I_1)

0.73057063989319271

In [25]:
#one liner
x__1 = [j+i*5 for i in range(B) for j in x_]
# print([(i, j) for i in range(5) for j in range(5,10)]) # to understand

In [121]:
w_ = np.array([2,3])
w__ = []
for i in range(B):
    w__.extend(w_+i*5)
# y_true[:, :, B:2]
w__
# y_true[:, :, w__].shape
a_ = np.sqrt(y_pred[:, :, w__])
b_ = np.sqrt(y_true[:, :, w__])
np.sum(((a_-b_)**2)*I_1)

0.059362076506496629

In [122]:
c_ = np.array([4])
c__ = []
for i in range(B):
    c__.extend(c_+i*5)
# y_true[:, :, B:2]
c__
a_ = y_true[:, :, c__]
b_ = y_pred[:, :, c__]
np.sum(((a_-b_)**2)*I_1)

0.73079649517707657

In [125]:
a_ = y_true[:, :, -C:]
b_ = y_pred[:, :, -C:]
np.sum(((a_-b_)**2)*I_2)

0.8497298726897492

### Final loss in Numpy

In [7]:
lamdba_1 = 5
lambda_2 = 0.05

In [71]:
def loss(y_pred, y_true):
    '''getting the first term'''
    I_1  = np.zeros((S,S,5*B+C), dtype='uint8')# i,j
    for i in range(B):
        t = (np.sum(y_true[:, :, i*5:(i+1)*5], axis=2)>0).astype(int)
        stacked = np.stack([t for ind in range(5)], axis=2)
        I_1[:, :, i*5:(i+1)*5] = stacked

    x_ = np.array([0,1])
    x__ = [j+i*5 for i in range(B) for j in x_]    
    a_ = y_pred[:, :, x__]*I_1[:, :, x__]
    b_ = y_true[:, :, x__]*I_1[:, :, x__]

    term1 = lamdba_1*np.sum((a_-b_)**2)
   
    
    '''second term'''
    w_ = np.array([2,3])
    w__ = [j+i*5 for i in range(B) for j in w_]
    c_ = np.sqrt(y_pred[:, :, w__])*I_1[:, :, w__]
    d_ = np.sqrt(y_true[:, :, w__])*I_1[:, :, w__]

    term2 = lamdba_1*np.sum((c_-d_)**2)
    
    
    '''third term'''
    C_ = np.array([4])
    C_ = [j+i*5 for i in range(B) for j in C_]
    e_ = y_pred[:, :, C_]*I_1[:, :, C_]
    f_ = y_true[:, :, C_]*I_1[:, :, C_]
    term3 = np.sum((e_-f_)**2)
    
    '''fourth term'''
    I_1_noobj = np.logical_not(I_1).astype(int)
    I_1_noobj[:, :, -C:] = np.zeros((S,S,2))

    g_ = y_pred[:, :, C_]*I_1_noobj[:, :, C_]
    h_ = y_true[:, :, C_]*I_1_noobj[:, :, C_]
    term4 = lambda_2*np.sum((g_-h_)**2)

    '''fifth term'''
    
    a = y_true[:, :, -C+1:] # 0th class is no object
    b = (np.sum(a, axis=2)>0).astype(int)
    I_2 = np.stack([b for ind in range(C)], axis=2)
    
    i_ = y_pred[:, :, -C:]*I_2
    j_ = y_true[:, :, -C:]*I_2
#     print(i_.shape, j_.shape)
    term5 = np.sum((i_-j_)**2)
    
    
    total_loss = term1 + term2 + term3 + term4 + term5
    return total_loss

In [72]:
loss(y_pred=y_pred, y_true=y_true)

10.91543715656109

## Final loss in Keras / tf

In [20]:
y_true_ = K.variable(y_true)
y_pred_ = K.variable(y_pred)

In [23]:
def loss_K():
    pass